<figure> 
<img src="../Imagenes/logo-final-ap.png"  width="80" height="80" align="left"/> 
</figure>

# <span style="color:blue"><left>Aprendizaje Profundo</left></span>

# <span style="color:red"><center>Redes GRU (Gated Recurrent Unit)</center></span>

<figure>
<center>
<img src="../Imagenes/640px-Gate_Apples_Rosemoor.jpg" width="600" height="600" align="center"/>
</center>
<figcaption>
<p style="text-align:center">Gate Apples Rosemoora</p>
</figcaption>
</figure>

Fuente: <a href="https://commons.wikimedia.org/wiki/File:Gate_Apples_Rosemoor.jpg">Kmtextor</a>, <a href="https://creativecommons.org/licenses/by-sa/4.0">CC BY-SA 4.0</a>, via Wikimedia Commons

## <span style="color:#4361EE">Coordinador</span>

- Campo Elías Pardo Turriago, cepardot@unal.edu.co 

## <span style="color:#4361EE">Profesores</span>

1. Alvaro  Montenegro, PhD, ammontenegrod@unal.edu.co
1. Camilo José Torres Jiménez, Msc, cjtorresj@unal.edu.co
1. Daniel  Montenegro, Msc, dextronomo@gmail.com 

## <span style="color:#4361EE">Asesora Medios y Marketing digital</span>

4. Maria del Pilar Montenegro, pmontenegro88@gmail.com
5. Jessica López Mejía, jelopezme@unal.edu.co

## <span style="color:#4361EE">Jefe Jurídica</span>

6. Paula Andrea Guzmán, guzmancruz.paula@gmail.com

## <span style="color:#4361EE">Coordinador Jurídico</span>

7. David Fuentes, fuentesd065@gmail.com

## <span style="color:#4361EE">Desarrolladores Principales</span>

8. Dairo Moreno, damoralesj@unal.edu.co
9. Joan Castro, jocastroc@unal.edu.co
10. Bryan Riveros, briveros@unal.edu.co
11. Rosmer Vargas, rovargasc@unal.edu.co
12. Venus Puertas, vpuertasg@unal.edu.co

## <span style="color:#4361EE">Expertos en Bases de Datos</span>

13. Giovvani Barrera, udgiovanni@gmail.com
14. Camilo Chitivo, cchitivo@unal.edu.co

## <span style="color:blue">Referencias</span>

1. [Introducción a Redes LSTM](Intro_LSTM.ipynb)
2. [Time Series Forecasting with LSTMs using TensorFlow 2 and Keras in Python](https://towardsdatascience.com/time-series-forecasting-with-lstms-using-tensorflow-2-and-keras-in-python-6ceee9c6c651/)
3. Basado en [Dive into Deep Learnig](https://d2l.ai/)

## <span style="color:blue">Contenido</span>

* [Introducción](#Introducción)
* [Funcionamiento de una red GRU](#Funcionamiento-de-una-red-GRU)
* [Puerta de actualización-update](#Puerta-de-actualización-update)
* [Puerta reinicio](#Puerta-reinicio)
* [Activación candidata](#Activación-candidata)
* [Actualización del estado recurrente](#Actualización-del-estado-recurrente)
* [Resumen de la matemática en la red GRU](#Resumen-de-la-matemática-en-la-red-GRU)


## <span style="color:blue">Introducción</span>



Una red neuronal recurrente (RNR) es una extensión de una red neuronal convencional,
que es capaz de manejar una entrada de secuencia de longitud variable. La RNR maneja la longitud variable de las secuencias mediante un estado oculto recurrente cuya activación en cada momento depende del estado anterior. 

Más formalmente, dada una secuencia $\mathbf{x}= (x_1, x_2, \ldots, x_T)$, la RNR actualiza su estado oculto recurrente $h_t$ mediante 


$$
h_t = \begin{cases}
0,  &\text{ si } t=0,\\
\phi(h_{t-1},x_t), &\text{en otro caso}.
\end{cases}
$$


Si $g$ es una función de activación  suave, como un sigmoide o una tangente hiperbólica, es común definir

$$
h_t = g(Wh_{t-1} + Ux_t+ b)
$$

En el tratamiento del lenguaje natural una RNR generativa calcula en cada paso una distribución de probabilidad para el siguiente elemento de la secuencia, dado su estado actual $h_t$. Este modelo generativo puede capturar una distribución sobre secuencias de longitud variable mediante el uso de un símbolo de salida especial para representar el final de la secuencia. La probabilidad se puede descomponer en

$$
p (x_1,\ldots, x_T) = p (x_1) p (x_2 | x_1) p (x_3 | x_1, x_2) \ldots p (x_T | x_1,\ldots, x_{T − 1}), 
$$

en donde el último elemento es un valor especial de final de secuencia. En la ecuación anterior cada probabilidad condicional es aproximada mediante

$$
p (x_t | x_1,\ldots, x_{t − 1}) = g (h_t)
$$

El problema con este modelo, es que el cálculo del gradiente tiende a volverse cero o a explotar. 

Dos líneas de trabajo fueron impulsadas por esta situación. Por un lado se inició la búsqueda de nuevas técnicas para el cálculo del gradiente en el proceso de optimización de la función de pérdida, y por el otro, el desarrollo de nuevos modelos de redes neuronales. 


La primera línea ha producido nuevas técnicas de optimización estocástica basados en el gradiente, que han sido usadas exitosamente en redes generales. 

La segunda línea llevó al desarrollo de las redes [LSTM](Intro_LSTM.ipynb), en las cuales la función de activación consiste en una transformación afinada seguida por una simple no linealidad de elementos mediante el uso de unidades de compuerta, [Hochreiter y Schmidhuber, 1997](https://www.bioinf.jku.at/publications/older/2604.pdf). 


Más recientemente, otro tipo de unidad recurrente, a la que nos referimos como una unidad recurrente cerrada (GRU), fue propuesta por [Cho et al. 2014](https://arxiv.org/pdf/1409.1259.pdf). Puede consultar una comparación entre LSTM y GRU en [Junyoung Chung et al., 2014](https://arxiv.org/pdf/1412.3555v1.pdf). De estas unidades recurrentes se ha demostrado que funcionan bien en tareas que requieren captura de dependencias a largo plazo.  Esas tareas incluyen, pero no se limitan a reconocimiento de voz, música,...

## <span style="color:blue">Funcionamiento de una red GRU</span>

Una red de unidad recurrente cerrada (GRU) permite que cada unidad recurrente capture de forma adaptativa dependencias de diferentes escalas de tiempo. De manera similar a la unidad LSTM, la GRU tiene compuertas que modulan el flujo de información dentro de la unidad.  Sin embargo, a diferencia de las redes LSTM no tiene celdas de memoria separadas. Las dos  imágenes muestran la estructura general de las compuertas GRU.



<figure>
<center>
<img src="../Imagenes/gru-1.svg" width="600" height="600" align="center"/>
</center>
<figcaption>
<p style="text-align:center">GRU</p>
</figcaption>
</figure>


Estos diagramas muestran la estructura matemática dentro de la compuerta en el tiempo $t$, con reinicio antes o después de de la multiplicación.

<figure>
<center>
<img src="../Imagenes/compuerta_GRU_after_false.jpg" width="800" height="600" align="center"/>
</center>
</figure>

Imagen: Alvaro Montenegro

+ reset_after: convención GRU (si aplicar la puerta de reinicio  antes o después de la multiplicación de matrices). False = "antes", True = "después" (este es el caso por defecto).

<figure>
<center>
<img src="../Imagenes/compuerta_GRU_after_true.jpg" width="800" height="600" align="center"/>
</center>
</figure>

Imagen: Alvaro Montenegro

La notación que usamos es bastante estándar. 

1. $+$ : indica suma de vectores
1. $\sigma$ : representa a la función de activación sigmoide
1. $\tanh$ : representa a la función de activación tangente hiperbólica.
1. $\ast$ : es producto componente a componente (producto de Hamard)

Una GRU tien dos tipos de puerta: actualización (update) y reinicio (reset).

## <span style="color:blue">Puerta de actualización-update</span>


Comenzamos calculando la salida de la compuerta de actualización. Supongamos que $z_t$ es la salida de la compuerta en el tiempo $t$, entonces tenemos que

$$
z_t = \sigma(W_z x_t + U_zh_{t-1} + b_z)
$$

en donde $W_z$ y $U_z$ son los pesos asociados a la nueva entrada $x_t$  y al estado oculto $h_{t-1}$  respectivamente y en donde $b_z$ es el respectivo vector de sesgo. En realidad esta formula es equivalente a la que utilizamos en la lección de LSTM, en donde concatenamos $x_t$ con $h_t$ y usamos una única matriz de pesos.


La puerta de actualización ayuda al modelo a determinar qué cantidad de la información pasada (de los pasos de tiempo anteriores) debe transmitirse al futuro, combinándola  con la nueva información. 


Eso es realmente poderoso porque el modelo puede decidir copiar toda la información del pasado y eliminar el riesgo de desvanecer del gradiente.

## <span style="color:blue">Puerta reinicio</span>

Esencialmente, esta puerta se utiliza para que el modelo decida qué cantidad de información pasada debe olvidar. La salida de la compuerta se calcula como

$$
r_t = \sigma(W_r x_t + U_rh_{t-1} + b_r),
$$

en donde $W_r$ y $U_r$ son pesos asociados $x_t$ (la nueva entrada) y $h_{t-1}$, (el estado oculto).

## <span style="color:blue">Activación candidata</span>

Veamos cómo afectarán exactamente las puertas al resultado final. Primero, comenzamos con el uso de la puerta de reinicio. Introducimos un nuevo contenido de memoria que utilizará la puerta de reinicio para almacenar la información relevante del pasado. Se calcula de la siguiente manera:


$$
\tilde{h}_t = \tanh(W_nx_t + U_n(r_t\ast h_{t-1})+b_n),
$$

en donde $W_n$ y $U_n$ son pesos asociados a las  $x$'s y a las $h$'s respectivamente.

## <span style="color:blue">Actualización del estado recurrente</span>

La activación de la GRU en tiempo $t$ es una interpolación lineal entre la activación previa ${h}_{t-1}$ y la activación candidata $\tilde{h}_t$ definida arriba. En símbolos tenemos:


$$
h_t = (1-z_t)\ast h_{t-1} + z_t \ast \tilde{h}_t 
$$

La siguiente imagen ilustra la arquitectura de la red.

<figure>
<center>
<img src="../Imagenes/plano_gru.jpg" width="800" height="600" align="center"/>
</center>
</figure>

Imagen: Alvaro Montenegro

## <span style="color:blue">Resumen de la matemática en la red GRU</span>

$$
z_t = \sigma(W_z x_t + U_zh_{t-1} + b_z)
$$

$$
r_t = \sigma(W_r x_t + U_rh_{t-1} + b_r),
$$

$$
\tilde{h}_t = \tanh(W_nx_t + U_n(r_t\odot h_{t-1})+b_n),
$$

$$
h_t = (1-z_t)\odot h_{t-1} + z_t \odot \tilde{h}_t 
$$

## <span style="color:blue">Computación  en una capa GRU de Keras</span>

Cualquier capa de Keras siempre espera un batch de datos. En el caso de una capa GRU Keras espera tensores 3D de la siguiente forma

+ [batch_size, window_len, num_features]


Por ejemplo un tensor de entrada de tamaño [32, 10, 8], la capa Keras lo interpreta como
- batch_size = 32, es decir 32 ejemplos.
- window_len = 10, es decir secuencias de entrada de tamaño 10. Por ejemplo en una serie de tiempo este es el tamaño de la ventana de entrada.
- num_features = 8, es decir la variable de entrada es de tamaño 8. Por ejemplo, en series de tiempo univariadas, feature = 1. En una serie multivariada con 8 variables, features = 8. En modelos de lenguaje natural feature = tamaño de representación de a cada token. Usualmente correspondería al tamaño del sumergimiento(embedding).

La salida de la capa corresponde al tamaño del estado oculto. Por ejemplo, si el estado oculto tiene tamaño, la salida de la capa es de tamaño [batch_size, 4].

### <span style="color:#4CC9F0">Ejemplo</span>

In [1]:
import tensorflow as tf
inputs = tf.random.normal([32,10,8])
gru = tf.keras.layers.GRU(4) # gru es una capa de tamaño de salidad 4.
output = gru(inputs)
print (output.shape)


2023-05-28 18:49:26.104248: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-28 18:49:43.079674: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-28 18:49:43.111826: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


(32, 4)


### <span style="color:#4CC9F0">Recibiendo toda la secuencia del valor del estado oculto</span>

En algunos casos es necesario disponer del valor del estado oculto para cada valor en la secuencia de entrada. Esta secuencia tiene tamaño [batch_size, window_len, output_size]
En el siguiente ejemplo se tiene que

- `return_sequences` son todos lo estados del estado oculto
- `return_state` es el último valor del estado oculto

In [2]:
gru = tf.keras.layers.GRU(4, return_sequences = True, return_state=True)
whole_seq_output, final_memory_state = gru(inputs)

In [3]:
print(whole_seq_output.shape)
print(final_memory_state.shape)


(32, 10, 4)
(32, 4)


In [4]:
whole_seq_output[:,-1] == final_memory_state

<tf.Tensor: shape=(32, 4), dtype=bool, numpy=
array([[ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True, 

### <span style="color:#4CC9F0">Extracción de los pesos de la capa</span>

In [6]:
weights = gru.get_weights()

In [7]:
len(weights)

3

In [8]:
weights[0].shape

(8, 12)

In [9]:
weights[1].shape

(4, 12)

In [10]:
weights[2].shape

(2, 12)

Note que los pesos de la capa GRU Están organizados de la siguiente forma

In [11]:
units = 4 # tamaño del estado oculto
W = gru.get_weights()[0]
W_z = W[:,:units]
W_r = W[:, units:units*2]
W_n = W[:, units*2:units*3]


U = gru.get_weights()[1]
U_z = U[:,:units]
U_r = U[:, units:units*2]
U_n = U[:, units*2:units*3]

b = gru.get_weights()[2] 
b_z = b[:units]
b_r = b[ units:units*2]
b_n = b[units*2:units*3]


[Regresar al inicio](#Contenido)